In [53]:
import os
import pandas as pd

# Iterating through every Excel forecast file and saving it into big df

In [56]:
# Get the file names from the folder
dirpath = 'Y:\\BI files\\Tableau\\Personal Tableau\\Adrian\\Projects\\Bestseller Forecast Bulks\\Forecast - Weekly update\\'
for _, _, files in os.walk(dirpath):
    filename_list = files

# Initialize empty DataFrame for appending
df_bulk = pd.DataFrame()

# Iterate through file list of forecast and append to df_bulk
for file in filename_list:
    
    # Read Excel into DataFrame
    df = pd.read_excel(io=dirpath + file,
                       sheet_name='INGRAM MICRO',
                       header=None)
    
    # Drop empty rows
    df.dropna(axis=0,
          how='all',
          subset=[0],
          inplace=True)
    
    # Set row labels as index and filter only by two columns
    df.set_index(0, inplace=True)
    df = df.loc[['Calendar date', 'Demand units: dropship TOTAL']]
    
    # Drop first column that contains only Totals
    df.drop(axis=1, columns=1)
    
    # Transpose df so that columns are rows and remove NaT rows
    df = df.transpose()
    df.dropna(inplace=True)
    
    # Add filename column slicing last 14 characters
    df['Filename'] = file[-14:]
    
    # Concatenate dataframe into big one
    df_bulk = pd.concat(objs=[df_bulk, df], axis=0, ignore_index=True)
    

# Clean up

In [57]:
# Add column where all non-numeric characters will be excluded
df_bulk['Num_Filename'] = df_bulk['Filename'].replace('[^0-9]', '', regex=True)

# Remove whitespace from "Demand units: dropship TOTAL" and drop NaN
df_bulk['Demand units: dropship TOTAL'].replace(' ', '', regex=True, inplace=True)
df_bulk['Demand units: dropship TOTAL'].dropna(inplace=True)

# Change data type to int64 and date
df_bulk = df_bulk.astype({'Num_Filename': 'int64', 'Demand units: dropship TOTAL': 'int64'})
df_bulk['Calendar date'] = pd.to_datetime(df_bulk['Calendar date'])

In [84]:
# Sort df by Num_Filename to have the freshest forecast at the top
df_bulk.sort_values(by=['Num_Filename', 'Calendar date'], 
                    axis='rows',
                    ascending=[False, True],
                    inplace=True)

df_bulk.drop_duplicates(subset=['Calendar date'], 
                        keep='first',
                        inplace=True,
                        ignore_index=True)

df_bulk['Calendar date'].describe() # check the date range of data

C:\Users\208896\AppData\Local\Temp/ipykernel_25500/1723218724.py:13: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_bulk['Calendar date'].describe()


count                    1106
unique                   1106
top       2021-10-25 00:00:00
freq                        1
first     2018-12-31 00:00:00
last      2022-01-09 00:00:00
Name: Calendar date, dtype: object

In [90]:
# Prepare data for export into Facebook Prophet

df_export = df_bulk[['Calendar date', 'Demand units: dropship TOTAL']].copy()
df_export.sort_values(by=['Calendar date'], ascending=True, inplace=True, ignore_index=True)
df_export.columns = ['ds', 'y']

# Write as csv
df_export.to_csv('bse_forecast.csv', 
                 sep=',',
                 index=False
                 )